In [1]:
from typing import List
import torch
import torch.nn as nn

import luigi

#from recommendation.utils import load_torch_model_training_from_task_dir
from recommendation.model.deep import DeepCTR
from recommendation.task.meta_config import RecommenderType
from recommendation.task.model.base import BaseTorchModelTraining, TORCH_ACTIVATION_FUNCTIONS, TORCH_WEIGHT_INIT, \
    TORCH_DROPOUT_MODULES
from recommendation.torch import MaskedZeroesLoss
import pandas as pd


/home/marlesson/anaconda3/envs/recommendation-system/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
from recommendation.utils import load_torch_model_training_from_task_id
from recommendation.task.model.deep import DeepTraining

In [3]:
model_cls = DeepTraining
task_dir  = "DeepTraining_relu____500_27ef511f18"

model = load_torch_model_training_from_task_id(model_cls, task_dir)
model._output_path = task_dir
model

DeepTraining(project=criteo, data_transformation=none, data_transformation_params={}, test_size=0.2, dataset_split_method=holdout, val_size=0.2, n_splits=5, split_index=0, data_frames_preparation_extra_params={}, sampling_strategy=none, balance_fields=[], sampling_proportions={}, use_sampling_in_validation=False, eq_filters={}, neq_filters={}, isin_filters={}, seed=42, device=cuda, mode=fit, lr_find_iterations=100, batch_size=500, epochs=20, optimizer=adam, optimizer_params={}, learning_rate=0.001, lr_scheduler=None, lr_scheduler_params={}, loss_function=bce_loss, loss_function_params={}, early_stopping_patience=10, early_stopping_min_delta=1e-06, generator_workers=0, pin_memory=False, metrics=["loss"], input_d_dim=13, input_c_dim=26, n_factors=10, dense_layers=[1024, 512, 256], dropout_prob=None, activation_function=relu, weight_init=lecun_normal, dropout_module=alpha)

In [4]:
model.get_trained_module()

2019-08-22 17:40:01,303 : INFO : Loaded ['luigi.cfg']


DeepCTR(
  (dense_block): Sequential(
    (0): Sequential(
      (0): Linear(in_features=273, out_features=1024, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): ReLU()
    )
  )
  (logistic): Sequential(
    (0): Linear(in_features=256, out_features=1, bias=True)
    (1): Sigmoid()
  )
  (emb_c1): Embedding(377, 10)
  (emb_c2): Embedding(472, 10)
  (emb_c3): Embedding(23346, 10)
  (emb_c4): Embedding(14437, 10)
  (emb_c5): Embedding(105, 10)
  (emb_c6): Embedding(12, 10)
  (emb_c7): Embedding(6332, 10)
  (emb_c8): Embedding(186, 10)
  (emb_c9): Embedding(4, 10)
  (emb_c10): Embedding(7724, 10)
  (emb_c11): Embedding(3356, 10)
  (emb_c12): Embedding(22061, 10)
  (emb_c13): Embedding(2555, 10)
  (emb_c14): Embedding(27, 10)
  (emb_c15): Embedding(4069, 10)
  (emb_c16): Embedding(18777, 10)
  (emb_c

In [5]:
len(model.test_dataset)

6042135

In [6]:
model.test_dataset._dense_columns

['I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9',
 'I10',
 'I11',
 'I12',
 'I13']

In [7]:
module = model.get_trained_module()

In [8]:
c1 = model.test_dataset._data_frame[model.test_dataset._dense_columns]
c2 = model.test_dataset._data_frame[model.test_dataset._categorical_columns]

In [9]:
c1.head()

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13
0,-1.0,29,50.0,5.0,7260.0,437.0,1.0,4.0,14.0,-1.0,1.0,0.0,6.0
1,27.0,17,45.0,28.0,2.0,28.0,27.0,29.0,28.0,1.0,1.0,-1.0,23.0
2,1.0,1,19.0,7.0,1.0,3.0,1.0,7.0,7.0,1.0,1.0,-1.0,2.0
3,4.0,1,-1.0,6.0,1051.0,134.0,4.0,35.0,72.0,1.0,1.0,-1.0,6.0
4,7.0,1,25.0,10.0,139.0,74.0,48.0,13.0,44.0,1.0,8.0,2.0,12.0


In [10]:
c2.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,8,113,0,1377,1,5,80,6,1,3887,...,7,174,1,2,0,1,10,938,4,384
1,1,259,0,0,1,1,636,20,1,490,...,1,559,1,2,0,1,2,4507,7,1367
2,23,29,2682,2081,1,1,3934,20,1,0,...,6,33,23,4,2472,1,2,1384,7,1047
3,4,323,2437,1904,1,6,619,2,1,0,...,2,784,2,3,2250,2,10,1278,2,3
4,4,37,1967,1570,1,1,855,2,1,1610,...,8,57,1,4,1833,2,1,1062,5,807


In [11]:
model.generate_submission_file()

100%|██████████| 12085/12085 [01:28<00:00, 136.67it/s]


In [12]:
a

NameError: name 'a' is not defined

In [ ]:
# data_loader = model.get_test_generator()
# data_loader._train = False
# module = module.to(torch.device("cpu"))
# for batch_idx, (xd, xc) in enumerate(data_loader):
#     print(batch_idx, module(xd, xc).reshape(-1))
#     

#module = module.to(torch.device("cpu"))
#module.forward(torch.from_numpy(c1.values), torch.from_numpy(c2.values))

In [ ]:
#model.generate_submission_file(module)

model.batch_size = 500
model.generator_workers = 0
model.device = 'cpu'


module      = model.get_trained_module()
data_loader = model.get_test_generator()

In [ ]:
from tqdm import tqdm
tqdm.pandas()
from recommendation.utils import chunks
import math
import numpy as np

df_test = model.test_dataset._data_frame
batch_size = 500

scores: List[float] = []
for indices in tqdm(chunks(range(len(df_test)), batch_size),
                    total=math.ceil(len(df_test) / batch_size)):
    rows: pd.DataFrame = df_test.iloc[indices]
        
    dense_columns = torch.tensor(rows[model.test_dataset._dense_columns].values,       dtype=torch.float64)
    cat_columns   = torch.tensor(rows[model.test_dataset._categorical_columns].values, dtype=torch.int64)
    batch_scores: torch.Tensor = module(dense_columns.to(torch.device("cuda")),
                                        cat_columns.to(torch.device("cuda")))
    scores.extend(batch_scores.detach().cpu().numpy().tolist())
    
scores = np.reshape(scores,-1)

df = pd.DataFrame({'Predicted': scores})
df['Id'] = [60000000+(x) for x in range(len(df))]
df.to_csv(os.path.join(output_path, "submission.csv"), index=False)

df.head()

In [ ]:
%%time

import os
output_path = '/media/marlesson/Backup/workspace/DeepFood/recommendation-system/output/models/DeepTraining/results/DeepTraining_relu____500_27ef511f18'


def generate_submission_file(model, data_loader):
    with torch.no_grad():
        model  = model#.to(torch.device("cpu"))
        probas = []
        
        for batch_idx, (xd, xc) in enumerate(data_loader):
             xd = xd.to(torch.device("cuda"))
             xc = xc.to(torch.device("cuda"))
             probas.extend(model(xd, xc).reshape(-1).cpu().numpy())
             print(batch_idx, xd.shape)

             if batch_idx > 10:
                break

        print(probas[:3])
        #df = pd.DataFrame({'Predicted': probas})
        #df['Id'] = [60000000+(x) for x in range(len(df))]
        #df.to_csv(os.path.join(output_path, "submission.csv"), index=False)
        

generate_submission_file(module, data_loader)        

In [ ]:
#model.device = 'cuda'

In [ ]:
model.batch_size, len(data_loader)

In [ ]:
# Loop over 
def step(model, inputs):
    xd, xc  = inputs # ignore label
    preds  = model(xd, xc).reshape(-1).numpy()
    return preds


def predict(model, dataloader):
    model  = model.to(torch.device("cpu", non_blocking=True))
    prediction_list = []
    for i, (xd, xc) in enumerate(dataloader):
        pred = step(model, (xd, xc))
        prediction_list.append(pred)
        print(i)
    return prediction_list

In [ ]:
with torch.no_grad():
    predict(module, data_loader)

In [ ]:
model.test_dataset._data_frame.shape[0]/500